In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "mohamedmagdybaioumy" # username from the json file
os.environ['KAGGLE_KEY'] = "5cb603829d59963059128846de4ad1a5" # key from the json 
!kaggle datasets download -d shayanfazeli/heartbeat #copied api

 90% 89.0M/98.8M [00:00<00:00, 195MB/s]
100% 98.8M/98.8M [00:00<00:00, 182MB/s]


In [2]:
!ls

heartbeat.zip  sample_data


In [3]:
!unzip heartbeat.zip

Archive:  heartbeat.zip
  inflating: mitbih_test.csv         
  inflating: mitbih_train.csv        
  inflating: ptbdb_abnormal.csv      
  inflating: ptbdb_normal.csv        


In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('/content/mitbih_train.csv', header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87554 entries, 0 to 87553
Columns: 188 entries, 0 to 187
dtypes: float64(188)
memory usage: 125.6 MB


In [7]:
df.drop_duplicates(keep=False, inplace=True)
df.shape

(87554, 188)

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, valid_index in split.split(df, df[187]):
  X = df.iloc[train_index]
  X_valid = df.iloc[valid_index]

In [9]:
X[187].value_counts() / len(X)

0.0    0.827734
4.0    0.073455
2.0    0.066102
1.0    0.025384
3.0    0.007324
Name: 187, dtype: float64

In [10]:
X_valid[187].value_counts() / len(X_valid)

0.0    0.827708
4.0    0.073440
2.0    0.066130
1.0    0.025413
3.0    0.007310
Name: 187, dtype: float64

In [11]:
y = X[187]
y_valid = X_valid[187]
X.pop(187)
X_valid.pop(187)

32695    0.0
57704    0.0
69505    0.0
16459    0.0
57169    0.0
        ... 
28885    0.0
55809    0.0
62061    0.0
50975    0.0
68323    0.0
Name: 187, Length: 17511, dtype: float64

In [12]:
import tensorflow 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import optimizers
import math

model = keras.Sequential()
model.add(keras.layers.Conv1D(64,6, activation="relu", input_shape=(187,1)))
model.add(keras.layers.Conv1D(128,6, activation="relu"))
model.add(keras.layers.Conv1D(256,6, activation="relu"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16,activation="relu"))
model.add(keras.layers.Dense(8,activation="relu"))
model.add(keras.layers.Dense(5, activation="softmax"))

# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=X.shape[1:]))
# model.add(keras.layers.Dense(50,activation="relu"))
# model.add(keras.layers.Dense(50,activation="relu"))
# model.add(keras.layers.Dense(5, activation="softmax"))

optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [16]:
import keras.callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

history=model.fit(X, y, epochs=15,
          validation_data=(X_valid,y_valid),
          callbacks = early_stopping_cb, batch_size=32)# class_weight=class_weights)

Epoch 1/15
2189/2189 [==============================] - 14s 6ms/step - loss: 0.0426 - accuracy: 0.9870 - val_loss: 0.0694 - val_accuracy: 0.9812
Epoch 2/15
2189/2189 [==============================] - 15s 7ms/step - loss: 0.0382 - accuracy: 0.9885 - val_loss: 0.0763 - val_accuracy: 0.9794
Epoch 3/15
2189/2189 [==============================] - 14s 6ms/step - loss: 0.0352 - accuracy: 0.9891 - val_loss: 0.0705 - val_accuracy: 0.9821
Epoch 4/15
2189/2189 [==============================] - 15s 7ms/step - loss: 0.0326 - accuracy: 0.9897 - val_loss: 0.0662 - val_accuracy: 0.9830
Epoch 5/15
2189/2189 [==============================] - 16s 7ms/step - loss: 0.0299 - accuracy: 0.9903 - val_loss: 0.0727 - val_accuracy: 0.9826
Epoch 6/15
2189/2189 [==============================] - 15s 7ms/step - loss: 0.0280 - accuracy: 0.9912 - val_loss: 0.0804 - val_accuracy: 0.9831
Epoch 7/15
2189/2189 [==============================] - 14s 6ms/step - loss: 0.0275 - accuracy: 0.9912 - val_loss: 0.0806 - val_ac

In [17]:
model.evaluate(X, y)

2189/2189 [==============================] - 7s 3ms/step - loss: 0.0218 - accuracy: 0.9933


[0.021829621866345406, 0.9932755827903748]

In [15]:
model.evaluate(X_valid, y_valid)

548/548 [==============================] - 2s 4ms/step - loss: 0.0675 - accuracy: 0.9822


[0.06753524392843246, 0.9822397232055664]